In [ ]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
from datetime import date, timedelta
import time
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_err = pd.read_csv('/content/drive/Shareddrives/dacon/404/train_err2.csv')
train_qui = pd.read_csv('/content/drive/Shareddrives/dacon/404/train_qui2.csv')
train_prob = pd.read_csv('/content/drive/Shareddrives/dacon/404/train_prob2.csv')

test_err = pd.read_csv('/content/drive/Shareddrives/dacon/404/test_err2.csv')
test_qui = pd.read_csv('/content/drive/Shareddrives/dacon/404/test_qui2.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9,10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3,10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
train_qui2 = train_qui.copy()
test_qui2 = test_qui.copy()

In [ ]:
test_qui2 = test_qui2.replace(',','', regex=True)
test_qui2.iloc[:,2:-1] = test_qui2.iloc[:,2:-1].astype(float)

In [ ]:
# train_qui2['user_id'] = train_qui2['user_id'] - 10000
# test_qui2['user_id'] = test_qui2['user_id'] - 30000
train_qui2['time'] = pd.to_datetime(train_qui['time'], format = '%Y-%m-%d %H:%M:%S')
test_qui2['time'] = pd.to_datetime(test_qui['time'], format = '%Y-%m-%d %H:%M:%S')

In [ ]:
# quality 자료 중에 가장 빈번하게 나온 상위 n개를 뽑아서 return
def quality_frequency(dataset):
  data = dataset.drop(['fwver', 'quality_3', 'quality_4', 'time'], axis = 1)
  names = list()
  data = data.fillna(-10)
  frequencies = pd.DataFrame()
  #ids = [0, 14997]
  ids = list(set(data.user_id.values))
  ids.sort()
  for id in tqdm(ids):
    fre_list = []
    temp = data[data['user_id'] == id]
    for i in range(1,12):
      fre = temp.iloc[1:,i].value_counts()
      one = fre.index[0]
      if one == 0:
        try:
          one = fre.index[1]
        except:
          one =0      
      fre_list.extend([one])
    most = pd.DataFrame(fre_list).transpose()
    frequencies = pd.concat([frequencies, most], axis = 0)
  k = list(data.columns)
  k.remove('user_id')
  frequencies.columns = k
  frequencies.index = ids
  return frequencies

In [ ]:
# # 에러 코드 중에 어떤 코드가 가장 자주 등장하는가.. 0은 정상으로 보이기 때문에 만약에 0이면 index[1]로 다음 수치를 try


train_qui_frequent = quality_frequency(train_qui2)
test_qui_frequent = quality_frequency(test_qui2)

In [ ]:
train_qui_frequent.to_csv("트레인_퀼리티_항목별_많이등장.csv")
test_qui_frequent.to_csv("테스트_퀼리티_항목별_많이등장.csv")

In [ ]:
# train_qui_frequent

In [ ]:
# 1일까지 발생한 오류 ~ 2일까지 발생한 오류 ~ 3일까지 ~~~ n일까지 발생한 오류(누적)을 0,1,2,3,4,5 칼럼에 넣어주는 함수
# 퀼리티 로그의 경우 문제가 생기면 문제 이후에 2시간 분량을 수집한다고 했으므로, 몇 번이나 수집되었는지 알면 문제가 몇 번 있었는지도 알 수 있다.
# time에 관한 EDA 결과, 에러코드 수집 시작시간 10월 31일 ~ 11월 30일까지

begins = date(2020, 10, 31).timetuple().tm_yday
ends = date(2020, 11, 30).timetuple().tm_yday


def days_sum_quality(data):
    # ids = [0, 14997]
    ids = list(set(data.user_id.values))
    ids.sort()
    ids_err = list()
    for id in tqdm(ids):
        so_far = 0
        id_err = list()
        dat = data[data['user_id'] == id]
        for days in range(begins, ends + 1):
            # day_errs = data[(data['time'].dt.dayofyear == days) & (data['user_id'] == id)].shape[0]/12
            day_errs = dat[dat['time'].dt.dayofyear == days].shape[0]/12
            so_far += day_errs
            id_err.extend([so_far])
        ids_err.append(id_err)
    err_sums = pd.DataFrame(ids_err, index = ids)
    return err_sums

In [ ]:
# test예시, 30000과 44997만 넣고 해 본 결과

In [ ]:
train_qui2

,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,time
0,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0,2020-11-29 09:00:00
1,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0,2020-11-29 09:00:00
2,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0,2020-11-29 09:00:00
3,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0,2020-11-29 09:00:00
4,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0,2020-11-29 09:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828619,24997,04.22.1778,0.0,0,0.0,0,0,1,0,0,0,0,17,0,0,2020-11-24 03:30:00
828620,24997,04.22.1778,0.0,0,0.0,0,0,0,0,0,0,0,17,0,0,2020-11-24 03:30:00
828621,24997,04.22.1778,0.0,0,0.0,0,0,3,0,0,0,0,17,0,0,2020-11-24 03:30:00
828622,24997,04.22.1778,0.0,0,0.0,0,0,0,0,0,0,0,17,0,0,2020-11-24 03:30:00


In [ ]:
train_quality_sum = days_sum_quality(train_qui2)
test_quality_sum = days_sum_quality(test_qui2)

In [ ]:
train_err2 = train_err.copy()
test_err2 = test_err.copy()

In [ ]:
train_err2['time'] = pd.to_datetime(train_err2['time'], format = '%Y-%m-%d %H:%M:%S')
# train_err2['user_id'] = train_err2['user_id'] - 10000
test_err2['time'] = pd.to_datetime(test_err2['time'], format = '%Y-%m-%d %H:%M:%S')
# test_err2['user_id'] = test_err2['user_id'] - 30000

In [ ]:
# 위에 quality와 동일한 과정인데, err 데이터를 가지고 해 본다.

def days_sum_err(data):
  # ids = [0, 14998]
  ids = list(set(data.user_id.values))
  ids.sort()
  ids_err = list()
  for id in tqdm(ids):
    so_far = 0
    id_err = list()    
    dat = data[data['user_id'] == id]
    for days in range(begins, ends + 1):
        #day_errs = data[(data['time'].dt.dayofyear == days) & (data['user_id'] == id)].shape[0]
        day_errs = dat[dat['time'].dt.dayofyear == days].shape[0]
        so_far += day_errs
        id_err.extend([so_far])
    ids_err.append(id_err)
  err_sums = pd.DataFrame(ids_err, index = ids)
  return err_sums

In [ ]:
test_err2[test_err2.user_id ==8]

,user_id,model_nm,fwver,errtype,errcode,time


In [ ]:
test_err_sum_by_day = days_sum_err(test_err2)

In [ ]:
train_err_sum_by_day = days_sum_err(train_err2)

In [ ]:
train_err2

,user_id,model_nm,fwver,errtype,errcode,time
0,10000,model_3,05.15.2138,15,1,2020-11-01 02:56:16
1,10000,model_3,05.15.2138,12,1,2020-11-01 03:03:09
2,10000,model_3,05.15.2138,11,1,2020-11-01 03:03:09
3,10000,model_3,05.15.2138,16,1,2020-11-01 05:05:14
4,10000,model_3,05.15.2138,4,0,2020-11-01 05:05:15
...,...,...,...,...,...,...
16554658,24999,model_3,05.15.2138,15,1,2020-11-30 16:30:51
16554659,24999,model_3,05.15.2138,16,1,2020-11-30 17:26:25
16554660,24999,model_3,05.15.2138,4,0,2020-11-30 17:26:25
16554661,24999,model_3,05.15.2138,4,0,2020-11-30 17:26:31


In [ ]:
train_err_sum_by_day

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
10000,0,11,20,38,43,53,62,82,89,94,...,230,235,240,245,254,265,277,295,308,317
10001,0,11,61,90,138,180,209,222,246,299,...,653,671,689,716,727,785,2237,2275,2337,2365
10002,0,10,23,36,51,60,68,85,96,108,...,232,248,251,257,268,273,289,289,297,306
10003,0,9,23,33,38,54,59,61,72,77,...,191,212,223,240,246,254,275,286,300,306
10004,0,25,46,95,123,134,162,180,209,229,...,587,603,621,644,671,690,714,732,755,777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,...,41,58,76,76,100,103,133,155,172,194
24996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
24997,0,9,43,77,141,163,194,218,248,274,...,550,579,621,654,695,718,748,779,813,826
24998,0,7,7,17,17,24,24,32,32,39,...,120,125,131,131,137,137,144,144,151,155


In [ ]:
test_err_sum_by_day.to_csv("테스트_에러발생_일일누적합계.csv")
train_err_sum_by_day.to_csv("트레인_에러발생_일일누적합계.csv")

train_quality_sum.to_csv("트레인_퀼리티로그_일일수집합계.csv")
test_quality_sum.to_csv("테스트_퀼리티로그_일일수집합계.csv")

In [ ]:
train_quality_sum

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0
10002,0.0,0.0,0.0,0.0,1.0,1.0,2.0,2.0,2.0,2.0,...,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,8.0
10004,0.0,0.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
10005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
10006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
24992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
24993,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0
24995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0


In [ ]:
# _err 자료에서 0번부터 41번 에러코드까지, 총 42가지의 에러가 각각 몇 번 발생했는지 카운트

def errcode_sum(data):
    counts = []
    #ids = [0,1,2,3,4]
    ids = list(set(data.user_id.values))
    ids.sort()
    
    for id in tqdm(ids):
        err_list = []
        errs = data[data.user_id == id].errcode.value_counts()

        first = errs.index[0]
        try:
            second = errs.index[1]
        except:
            second = -1

        err_list.extend([first, second]) #, second, third])
        counts.append(err_list)
    frames = pd.DataFrame(counts, columns = ['Most_Err_Code', 'Second_Err_Code'], index = ids)
    return frames

In [ ]:
train_err_sum = errcode_sum(train_err2)

In [ ]:
test_err_sum = errcode_sum(test_err2)

In [ ]:
train_err_sum

,Most_Err_Code,Second_Err_Code
10000,1,0
10001,1,standby
10002,1,0
10003,1,0
10004,1,connection timeout
...,...,...
24995,1,0
24996,1,4
24997,1,0
24998,1,0


In [ ]:
# train_err과 test_err에서 errcode를 측정한 것이 errcode_sum 함수라면
# err_counter 함수는  42가지의 "ERR TYPE"이 각 id 별로 얼마나 나왔는지 파악하여 정리하는 함수.

def err_counter(data):
    #ids = [10000,24999]
    ids = list(set(data.user_id.values))
    err_count = []
    ids.sort()
    for id in tqdm(ids):
        err_freq = []
        temp = data[data.user_id == id].errtype.value_counts()
        for error in range(1, 43):
            err = list(temp[temp.index == error])
            if err == []:
                err_freq.extend([0])
            else:
                err_freq.extend(err)
        err_count.append(err_freq)
    counter = pd.DataFrame(err_count, columns = [i for i in range(1,43)], index = ids)
    return counter

,user_id,model_nm,fwver,errtype,errcode,time
0,0,model_3,05.15.2138,15,1,2020-11-01 02:56:16
1,0,model_3,05.15.2138,12,1,2020-11-01 03:03:09
2,0,model_3,05.15.2138,11,1,2020-11-01 03:03:09
3,0,model_3,05.15.2138,16,1,2020-11-01 05:05:14
4,0,model_3,05.15.2138,4,0,2020-11-01 05:05:15
...,...,...,...,...,...,...
16554658,14999,model_3,05.15.2138,15,1,2020-11-30 16:30:51
16554659,14999,model_3,05.15.2138,16,1,2020-11-30 17:26:25
16554660,14999,model_3,05.15.2138,4,0,2020-11-30 17:26:25
16554661,14999,model_3,05.15.2138,4,0,2020-11-30 17:26:31


In [ ]:
train_err_sum.to_csv("트레인_에러_에러코드별_발생횟수.csv")
test_err_sum.to_csv("테스트_에러_에러코드별_발생횟수.csv")

In [ ]:
train_err_counter = err_counter(train_err2)
test_err_counter = err_counter(test_err2)

In [ ]:
train_err_counter.to_csv("트레인_에러_에러타입별_발생횟수.csv")
test_err_counter.to_csv("테스트_에러_에러타입별_발생횟수.csv")

,user_id,model_nm,fwver,errtype,errcode,time
0,0,model_3,05.15.2138,15,1,2020-11-01 02:56:16
1,0,model_3,05.15.2138,12,1,2020-11-01 03:03:09
2,0,model_3,05.15.2138,11,1,2020-11-01 03:03:09
3,0,model_3,05.15.2138,16,1,2020-11-01 05:05:14
4,0,model_3,05.15.2138,4,0,2020-11-01 05:05:15
...,...,...,...,...,...,...
16554658,14999,model_3,05.15.2138,15,1,2020-11-30 16:30:51
16554659,14999,model_3,05.15.2138,16,1,2020-11-30 17:26:25
16554660,14999,model_3,05.15.2138,4,0,2020-11-30 17:26:25
16554661,14999,model_3,05.15.2138,4,0,2020-11-30 17:26:31
